In [1]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to C:\Users\Jack
[nltk_data]     Ellis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv(r'..\..\..\data\raw_data\seniorCanadaPostings2.csv',encoding='utf-8')

In [4]:
stpwords = stopwords.words('english')
stpwords.append('')
print(stpwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
allDescriptions = []
allWords = []

count = 0
for description in df.Description: 
    sentences = description.split('\n')
    words = []
    for sentence in sentences:
        if sentence is 'QUALIFICATIONS':
            print("HEY")
        newWords = sentence.split(' ')
        words += newWords
    finalWords = []
    for i in range(len(words)):
        #cleanWord = ''.join(filter(str.isalpha, words[i])).lower()
        cleanWord = re.sub('[\W_]+', '', words[i])
        if cleanWord not in stpwords:
            if cleanWord.isnumeric():
                if i+1<len(words):
                    secondWord = re.sub('[\W_]+', '', words[i+1])
                    totalWord = cleanWord + ' ' + secondWord
                    finalWords.append(totalWord)
            else:
                finalWords.append(cleanWord)
    allDescriptions.append(finalWords)
    allWords += finalWords
allWords = list(dict.fromkeys(allWords))

In [6]:
allDescriptionsBOW = {i:0 for i in allWords}

In [7]:
for description in allDescriptions:
    for word in description:
        allDescriptionsBOW[word] += 1

In [8]:
keys = list(allDescriptionsBOW.keys())
values = list(allDescriptionsBOW.values())

myList = []
for i in range(len(keys)):
    myList.append([keys[i],values[i]])

In [9]:
myList = sorted(myList,key=lambda l:l[1], reverse=True)

In [10]:
myList

[['data', 6301],
 ['experience', 2621],
 ['team', 2498],
 ['de', 2402],
 ['work', 2373],
 ['business', 2158],
 ['We', 1994],
 ['Experience', 1818],
 ['skills', 1621],
 ['Data', 1611],
 ['et', 1446],
 ['You', 1388],
 ['development', 1365],
 ['The', 1309],
 ['years', 1123],
 ['learning', 1108],
 ['working', 1100],
 ['analysis', 1100],
 ['des', 1080],
 ['solutions', 1069],
 ['new', 972],
 ['including', 968],
 ['environment', 919],
 ['management', 884],
 ['tools', 883],
 ['support', 845],
 ['research', 841],
 ['technical', 839],
 ['analytics', 806],
 ['knowledge', 783],
 ['product', 777],
 ['using', 776],
 ['les', 776],
 ['teams', 758],
 ['people', 753],
 ['ability', 751],
 ['role', 741],
 ['process', 733],
 ['models', 716],
 ['machine', 716],
 ['design', 706],
 ['software', 694],
 ['Our', 691],
 ['projects', 680],
 ['la', 679],
 ['information', 667],
 ['Strong', 650],
 ['products', 643],
 ['related', 638],
 ['technology', 636],
 ['science', 631],
 ['A', 623],
 ['help', 622],
 ['en', 607],

In [11]:
with open(r'..\..\..\data\clean_data\NLP\listfile.txt', 'w') as filehandle:
    for listitem in myList:
        filehandle.write('%s\n' % listitem)

UnicodeEncodeError: 'charmap' codec can't encode character '\u014d' in position 3: character maps to <undefined>